### 2.2.1 Example: testing for COVID-19 from Kevin Murphy's Probabilistic Machine Learning

Source: https://probml.github.io/pml-book/book1.html

You get tested for COVID-19 with the following parameters:

- Prevalence of Disease in Area: P(C = 1) = 0.1
- Sensitivity / True Positive Rate: P(T = 1 | C = 1)  = 87.5%
- Specificity / True Negative Rate: P(T = 0 | C = 0) = 97.5%

where C = 1 represents a COVID infection and T = 1 represents a positive test result. 

### Analytical Solution

What is the probability of being infected with COVID given a positive test result?

$P(C = 1 | T = 1) = \frac{P(T = 1 | C = 1) \times P(C = 1)}{P(T = 1 | C = 1) \times P(C = 1) + P(T = 1 | C = 0) \times P(C = 0)} = \frac{TPR \times prior}{TPR \times prior + FPR \times (1 - prior)} = \frac{0.875 \times 0.1}{0.875 \times 0.1 + 0.125 \times 0.9} = 0.795$

What is the probability of being infected with COVID given a negative test result?


$P(C = 1 | T = 0) = \frac{P(T = 0 | C = 1) \times P(C = 1)}{P(T = 0 | C = 1) \times P(C = 1) + P(T = 0 | C = 0) \times P(C = 0)} = \frac{FNR \times prior}{FNR \times prior + TNR \times (1 - prior)} = \frac{0.025 \times 0.1}{0.025 \times 0.1 + 0.975 \times 0.1} = 0.014$

### PyMC Simulation

In [1]:
import pymc3 as pm

def covid_model(observed: int, prevalence: float, sensitivity: float, specificity: float, iterations: int) -> float:

    with pm.Model() as model:
        # prevalence of disease / prior
        have_covid = pm.Bernoulli('have_covid', prevalence)
        
        # probability of testing positive = COVID + TPR + (1 - COVID) * FPR
        test_positive_prob = have_covid * sensitivity + (1 - have_covid) * (1 - specificity)
        
        # test result observed positive or negative
        test_positive = pm.Bernoulli('test_positive', test_positive_prob, observed = observed)
        
        disease_trace = pm.sample(iterations, return_inferencedata=False)
    return disease_trace['have_covid'].mean()

covid_when_pos = covid_model(observed = 1, prevalence = 0.1, sensitivity = 0.875, specificity = 0.975, iterations = 100_000)
covid_when_neg = covid_model(observed = 0, prevalence = 0.1, sensitivity = 0.875, specificity = 0.975, iterations = 100_000)

print(f"Probability of Infection if Tested Positive:\t{covid_when_pos:.04f}\n"\
      f"Probability of Infection if Tested Negative:\t{covid_when_neg:.04f}")

Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [have_covid]


Sampling 4 chains for 1_000 tune and 100_000 draw iterations (4_000 + 400_000 draws total) took 42 seconds.
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [have_covid]


Sampling 4 chains for 1_000 tune and 100_000 draw iterations (4_000 + 400_000 draws total) took 49 seconds.


Probability of Infection if Tested Positive:	0.7952
Probability of Infection if Tested Negative:	0.0141


In [2]:
import sys
print(f"PyMC3 version: {pm.__version__}")
print(f"Python version: {sys.version.split(' ')[0]}")

PyMC3 version: 3.10.0
Python version: 3.8.2
